In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV 

In [ ]:
from __future__ import print_function
import time
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [ ]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot

In [ ]:
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from scipy import interp
from sklearn.metrics import roc_auc_score
from sklearn import metrics

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

In [ ]:
import lime
from lime import lime_tabular

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [ ]:
data3000 = pd.read_csv('Cancer_3000features.csv')

In [ ]:
data3000.info()

In [ ]:
np.random.seed(42)

In [ ]:
data3000.head()

In [ ]:
data3000.target.nunique()

In [ ]:
data3000.target.unique()

In [ ]:
data3000.Labels.value_counts()

In [ ]:
# delete rows which have <= 4 labels
indexNames = data3000[(data3000['Labels'] == 'SARC.Normal')].index
# Delete these row indexes from dataFrame
data3000.drop(indexNames , inplace=True)

In [ ]:
indexNames = data3000[(data3000['Labels'] == 'CESC.Normal')].index
# Delete these row indexes from dataFrame
data3000.drop(indexNames , inplace=True)

In [ ]:
indexNames = data3000[(data3000['Labels'] == 'PCPG.Normal')].index
# Delete these row indexes from dataFrame
data3000.drop(indexNames , inplace=True)

In [ ]:
indexNames = data3000[(data3000['Labels'] == 'THYM.Normal')].index
# Delete these row indexes from dataFrame
data3000.drop(indexNames , inplace=True)

In [ ]:
indexNames = data3000[(data3000['Labels'] == 'STAD.Normal')].index
# Delete these row indexes from dataFrame
data3000.drop(indexNames , inplace=True)

In [ ]:
indexNames = data3000[(data3000['Labels'] == 'SKCM.Normal')].index
# Delete these row indexes from dataFrame
data3000.drop(indexNames , inplace=True)

In [ ]:
data3000.Labels.value_counts()

In [ ]:
data3000.head()

In [ ]:
data3000['target'] = data3000['Labels']

In [ ]:
data3000.drop('Labels', axis = 1, inplace = True)

In [ ]:
data3000.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
data3000.shape

In [ ]:
data3000.to_csv('Cancer_3000features.csv')

In [ ]:
X = data3000.iloc[:, :-1].values
y = data3000.iloc[:, -1].values

In [ ]:
y

In [ ]:
data3000["target"].head(2)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, shuffle=True, random_state = 101)

In [ ]:
flag = True
while flag:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, shuffle=True)
    if (pd.Series(y_test).nunique() == pd.Series(y_train).nunique()):
            flag = False

In [ ]:
pd.Series(y_train).nunique()

In [ ]:
pd.Series(y_test).nunique()

In [ ]:
print(len(X_train))
print(len(X_test))
print(X_train)

In [ ]:
labels = data3000['target']

In [ ]:
#data3000.drop('target', axis = 1, inplace = True)

In [ ]:
from sklearn.preprocessing import StandardScaler
feature_scaler = StandardScaler()
X_train = feature_scaler.fit_transform(X_train)
X_test = feature_scaler.transform(X_test)

In [ ]:
folds = KFold(5,shuffle = True, random_state=0)
classifier = SVC(probability=True, kernel= 'rbf', random_state=42)
n_classes = data3000.target.nunique()

In [ ]:
n_classes

In [ ]:
for i, (train, test) in enumerate(folds.split(X_train, y_train)):

    classifier.fit(X_train[train], y_train[train])
    
    y_pred = classifier.predict(X_train[test])
    
    conf_mat = confusion_matrix(y_train[test], y_pred)
    print (conf_mat)

    precision = precision_score(y_train[test], y_pred, average='micro')
    print('Precision micro: %.3f' % precision)
    precision = precision_score(y_train[test], y_pred, average='macro')
    print('Precision macro: %.3f' % precision)
    recall = recall_score(y_train[test], y_pred, average='micro')
    print('Recall micro: %.3f' % recall)
    recall = recall_score(y_train[test], y_pred, average='macro')
    print('Recall macro: %.3f' % recall)
    print(accuracy_score(y_train[test], y_pred))
    print(classification_report(y_train[test], y_pred))

In [ ]:
y_pred_cv = classifier.predict(X_test)
print(classification_report(y_test, y_pred_cv))

In [ ]:
precision_score(y_test, y_pred_cv, average='micro')

In [ ]:
recall_score(y_test, y_pred_cv, average='micro')

In [ ]:
f1_score(y_test, y_pred_cv, average='micro')

In [ ]:
len(y_test)

In [ ]:
roc_auc_score(y_test, classifier.predict_proba(X_test), multi_class='ovr')

In [ ]:
(uniquet, countst) = np.unique(y_test, return_counts=True)
frequenciest = np.asarray((uniquet, countst)).T
frequenciest

In [ ]:
#unique values in y_train that are not in y_test
np.setdiff1d(y_train, y_test)

In [ ]:
#unique values in y_test that are not in y_train
np.setdiff1d(y_test, y_train)

# LIME

In [ ]:
training_columns = data3000.iloc[:, :-1]

In [ ]:
training_columns.columns

In [ ]:
explainer = lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=training_columns.columns,
    class_names=list(np.unique(labels)),
    mode='classification'
)

In [ ]:
exp = explainer.explain_instance(
    data_row=X_test[1], 
    predict_fn=classifier.predict_proba, num_features=10, top_labels=2
)

exp.show_in_notebook(show_table=True)

In [ ]:
exp = explainer.explain_instance(
    data_row=X_test[2],
    predict_fn=classifier.predict_proba, num_features=10, top_labels=2
)

exp.show_in_notebook(show_table=True)

In [ ]:
exp = explainer.explain_instance(
    data_row=X_test[20],
    predict_fn=classifier.predict_proba, num_features=10, top_labels=2
)

exp.show_in_notebook(show_table=True)

In [ ]:
print(exp.available_labels())

In [ ]:
print(y_test[1])
print(y_pred_cv[1])
print(y_test[2])
print(y_pred_cv[2])

print(y_test[20])
print(y_pred_cv[20])

In [ ]:
X_test[1]

In [ ]:
exp = explainer.explain_instance(
    data_row=X_test[1], 
    predict_fn=classifier.predict_proba , num_features=6, top_labels=2
)

exp.show_in_notebook(show_table=True)

In [ ]:
exp = explainer.explain_instance(
    data_row=X_test[1], 
    predict_fn=classifier.predict_proba, num_features=6 , labels=[1]
)

#print('Predicted class =', labels[classifier.predict(X_test[1])])
#print('True class: %s' % labels[y_test[1]])

print ('Explanation for class %s' % labels[1])
print ('\n'.join(map(str, exp.as_list(label=1))))